![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/5.Spark_OCR.ipynb.ipynb)

# Spark OCR

### .. will be enriched ... work in progress ...

In [4]:
from sparkocr.databricks import create_init_script_for_tesseract
create_init_script_for_tesseract()

In [5]:
from sparkocr.databricks import display_images, OCR_MODEL_DIR


In [6]:
import sparkocr
import sys
from pyspark.sql import SparkSession
from sparkocr import start
import os

spark = start(secret=os.environ['JSL_OCR_SECRET'])
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.164.230.13:7077 
 AppName 
 Databricks Shell

In [7]:
%sh

wget 'http://www.asx.com.au/asxpdf/20171103/pdf/43nyyw9r820c6r.pdf'

URL transformed to HTTPS due to an HSTS policy
--2020-04-17 11:27:59-- https://www.asx.com.au/asxpdf/20171103/pdf/43nyyw9r820c6r.pdf
Resolving www.asx.com.au (www.asx.com.au)... 203.15.147.66
Connecting to www.asx.com.au (www.asx.com.au)|203.15.147.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212973 (208K) [application/pdf]
Saving to: ‘43nyyw9r820c6r.pdf.1’

 0K .......... .......... .......... .......... .......... 24% 297K 1s
 50K .......... .......... .......... .......... .......... 48% 297K 0s
 100K .......... .......... .......... .......... .......... 72% 26.0M 0s
 150K .......... .......... .......... .......... .......... 96% 298K 0s
 200K ....... 100% 29.5M=0.5s

2020-04-17 11:28:00 (410 KB/s) - ‘43nyyw9r820c6r.pdf.1’ saved [212973/212973]

In [8]:
dbutils.fs.cp("file:/databricks/driver/43nyyw9r820c6r.pdf", "dbfs:/")

Out[4]: True

In [9]:
dbutils.fs.mv("dbfs:/43nyyw9r820c6r.pdf", "dbfs:/sample_pdf_ocr.pdf")

Out[5]: True

In [10]:
import base64
from sparkocr.transformers import *
from pyspark.ml import PipelineModel

def pipeline():

    # Transforrm PDF document to struct image format
    pdf_to_image = PdfToImage()
    pdf_to_image.setInputCol("content")
    pdf_to_image.setOutputCol("image")
    pdf_to_image.setResolution(200)

    # Run tesseract OCR
    ocr = TesseractOcr()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    ocr.setTessdata(OCR_MODEL_DIR)
    
    pipeline = PipelineModel(stages=[
        pdf_to_image,
        ocr
    ])
    
    return pipeline

In [11]:
pdf = 'sample_pdf_ocr.pdf'
pdf_example_df = spark.read.format("binaryFile").load(pdf).cache().repartition(4)
pdf_example_df.show(3)

+--------------------+-------------------+------+--------------------+
 path| modificationTime|length| content|
+--------------------+-------------------+------+--------------------+
dbfs:/sample_pdf_...|2020-04-17 11:28:06|212973|[25 50 44 46 2D 3...|
+--------------------+-------------------+------+--------------------+

In [12]:
display(pdf_example_df)

path,modificationTime,length,content
dbfs:/sample_pdf_ocr.pdf,2020-04-17T11:28:06.000+0000,212973,JVBERi0xLjUNJeLjz9MNCjUgMCBvYmoNPDwvRSAyMTIyMjEvSCBbIDEwODEgMTQ1IF0vTCAyMTI5NzMvTGluZWFyaXplZCAxL04gMS9PIDgvVCAyMTI1NTM+Pg1lbmRvYmoNICAgICAgICAgICAgICAgICAgICAgICAgICAgDQ0= (truncated)


In [13]:
result = pipeline().transform(pdf_example_df).cache()


In [14]:
result.select("pagenum","text", "confidence").show()

+-------+--------------------+-----------------+
pagenum| text| confidence|
+-------+--------------------+-----------------+
 0|O’igitalx

ASX AN...|88.50514560275607|
+-------+--------------------+-----------------+

In [15]:
result.select("pagenum","text", "confidence").show()

+-------+--------------------+-----------------+
pagenum| text| confidence|
+-------+--------------------+-----------------+
 0|O’igitalx

ASX AN...|88.50514560275607|
+-------+--------------------+-----------------+

In [16]:
result.select("text").collect()

Out[16]: [Row(text='O’igitalx\n\nASX ANNOUNCEMENT\n3 November 2017\n\nNotice Pursuant to Paragraph 708A(5)(e) of the Corporations Act\n2001 ("Act")\n\nDigitalX Limited (ASX:DCC) (DCC or the Company) confirms that the Company has today\nissued 620,000 Fully Paid Ordinary Shares (Shares) upon exercise of 620,000 Unlisted\nOptions exercisable at $0.0324 Expiring 14 September 2019 and 3,725,000 Shares upon\nexercise of 3,725,000 Unlisted Incentive Options exercisable at $0.08 expiring 10 February\n2018.\n\nThe Act restricts the on-sale of securities issued without disclosure, unless the sale is exempt\nunder section 708 or 708A of the Act. By giving this notice, a sale of the Shares noted above\nwill fall within the exemption in section 708A(5) of the Act.\n\nThe Company hereby notifies ASX under paragraph 708A(5)(e) of the Act that:\n(a) the Company issued the Shares without disclosure to investors under Part 6D.2 of\nthe Act;\n(b) as at the date of this notice, the Company has complied with the provisions of Chapter\n2M of the Act as they apply to the Company, and section 674 of the Act; and\n(c) as at the date of this notice, there is no information:\na. that has been excluded from a continuous disclosure notice in accordance with\nthe ASX Listing Rules; and\nb. that investors and their professional advisers would reasonably require for the\npurpose of making an informed assessment of:\n[ the assets and liabilities, financial position and performance, profits and\nlosses and prospects of the Company; or\nii. the rights and liabilities attaching to the relevant Shares.\n\nDigitalX Limited\n\nLeigh Travers\nChief Executive Officer\nT: +61 439 376 847\n\nAbout DigitalX\n\nDigitalX is a Blockchain technology Company with offices in Perth and New York. DigitalX\nprovides Blockchain consulting services, ICO advisory services and software development.\nPartners can use DigitalX’s technology expertise to deliver innovative products to global\nmarkets.\n')]

In [17]:
print("\n".join([row.text for row in result.select("text").collect()]))


O’igitalx

ASX ANNOUNCEMENT
3 November 2017

Notice Pursuant to Paragraph 708A(5)(e) of the Corporations Act
2001 ("Act")

DigitalX Limited (ASX:DCC) (DCC or the Company) confirms that the Company has today
issued 620,000 Fully Paid Ordinary Shares (Shares) upon exercise of 620,000 Unlisted
Options exercisable at $0.0324 Expiring 14 September 2019 and 3,725,000 Shares upon
exercise of 3,725,000 Unlisted Incentive Options exercisable at $0.08 expiring 10 February
2018.

The Act restricts the on-sale of securities issued without disclosure, unless the sale is exempt
under section 708 or 708A of the Act. By giving this notice, a sale of the Shares noted above
will fall within the exemption in section 708A(5) of the Act.

The Company hereby notifies ASX under paragraph 708A(5)(e) of the Act that:
(a) the Company issued the Shares without disclosure to investors under Part 6D.2 of
the Act;
(b) as at the date of this notice, the Company has complied with the provisions of Chapter
2M of the Act as they apply to the Company, and section 674 of the Act; and
(c) as at the date of this notice, there is no information:
a. that has been excluded from a continuous disclosure notice in accordance with
the ASX Listing Rules; and
b. that investors and their professional advisers would reasonably require for the
purpose of making an informed assessment of:
[ the assets and liabilities, financial position and performance, profits and
losses and prospects of the Company; or
ii. the rights and liabilities attaching to the relevant Shares.

DigitalX Limited

Leigh Travers
Chief Executive Officer
T: +61 439 376 847

About DigitalX

DigitalX is a Blockchain technology Company with offices in Perth and New York. DigitalX
provides Blockchain consulting services, ICO advisory services and software development.
Partners can use DigitalX’s technology expertise to deliver innovative products to global
markets.

In [18]:
display_images(PdfToImage().setOutputCol("image").transform(pdf_example_df), limit=3)


Origin: dbfs:/sample_pdf_ocr.pdf 
 Resolution: 300 dpi 
 Width: 2479 px 
 Height: 3508 px 
 Mode: Greyscale 
 Number of channels: 1

In [19]:
result = pipeline().transform(pdf_example_df).cache()


In [20]:
display(result.select("pagenum", "text", "confidence"))


pagenum,text,confidence
0,"O’igitalx ASX ANNOUNCEMENT 3 November 2017 Notice Pursuant to Paragraph 708A(5)(e) of the Corporations Act 2001 (""Act"") DigitalX Limited (ASX:DCC) (DCC or the Company) confirms that the Company has today issued 620,000 Fully Paid Ordinary Shares (Shares) upon exercise of 620,000 Unlisted Options exercisable at $0.0324 Expiring 14 September 2019 and 3,725,000 Shares upon exercise of 3,725,000 Unlisted Incentive Options exercisable at $0.08 expiring 10 February 2018. The Act restricts the on-sale of securities issued without disclosure, unless the sale is exempt under section 708 or 708A of the Act. By giving this notice, a sale of the Shares noted above will fall within the exemption in section 708A(5) of the Act. The Company hereby notifies ASX under paragraph 708A(5)(e) of the Act that: (a) the Company issued the Shares without disclosure to investors under Part 6D.2 of the Act; (b) as at the date of this notice, the Company has complied with the provisions of Chapter 2M of the Act as they apply to the Company, and section 674 of the Act; and (c) as at the date of this notice, there is no information: a. that has been excluded from a continuous disclosure notice in accordance with the ASX Listing Rules; and b. that investors and their professional advisers would reasonably require for the purpose of making an informed assessment of: [ the assets and liabilities, financial position and performance, profits and losses and prospects of the Company; or ii. the rights and liabilities attaching to the relevant Shares. DigitalX Limited Leigh Travers Chief Executive Officer T: +61 439 376 847 About DigitalX DigitalX is a Blockchain technology Company with offices in Perth and New York. DigitalX provides Blockchain consulting services, ICO advisory services and software development. Partners can use DigitalX’s technology expertise to deliver innovative products to global markets.",88.50514560275607


In [21]:
%sh
wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/ocr/datasets/news.2B/5/8205_003.2B.tif

--2020-04-17 11:35:46-- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/ocr/datasets/news.2B/5/8205_003.2B.tif
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.77.54
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.77.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26988 (26K) [image/tiff]
Saving to: ‘8205_003.2B.tif’

 0K .......... .......... ...... 100% 329K=0.08s

2020-04-17 11:35:46 (329 KB/s) - ‘8205_003.2B.tif’ saved [26988/26988]

## Image parsing with Spark OCR

In [23]:
def pipeline():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")

    # Run tesseract OCR
    ocr = TesseractOcr()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    ocr.setTessdata(OCR_MODEL_DIR)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        ocr
    ])
    
    return pipeline

In [24]:
dbutils.fs.mv("file:/databricks/driver/8205_003.2B.tif", "dbfs:/sample_image_OCR.tif")

Out[19]: True

In [25]:
%sh
OCR_DIR=/dbfs/tmp/ocr
if [ ! -d "$OCR_DIR" ]; then
    mkdir $OCR_DIR
    cd $OCR_DIR
    wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/ocr/datasets/images.zip
    unzip images.zip
fi

In [26]:
display(dbutils.fs.ls("dbfs:/tmp/ocr/images/"))


path,name,size
dbfs:/tmp/ocr/images/.DS_Store,.DS_Store,10244
dbfs:/tmp/ocr/images/8200_006.2B.tif,8200_006.2B.tif,116158
dbfs:/tmp/ocr/images/8200_006.2B.txt,8200_006.2B.txt,3429
dbfs:/tmp/ocr/images/8200_006.2B.uzn,8200_006.2B.uzn,177
dbfs:/tmp/ocr/images/8200_014.2B.tif,8200_014.2B.tif,116116
dbfs:/tmp/ocr/images/8200_014.2B.txt,8200_014.2B.txt,3309
dbfs:/tmp/ocr/images/8200_014.2B.uzn,8200_014.2B.uzn,87
dbfs:/tmp/ocr/images/8200_015.2B.tif,8200_015.2B.tif,80978
dbfs:/tmp/ocr/images/8200_015.2B.txt,8200_015.2B.txt,2291
dbfs:/tmp/ocr/images/8200_015.2B.uzn,8200_015.2B.uzn,116


In [27]:

images_path = "/tmp/ocr/images/*.tif"
images_example_df = spark.read.format("binaryFile").load(images_path).cache()
display(images_example_df)

path,modificationTime,length,content
dbfs:/tmp/ocr/images/8230_022.2B.tif,2020-03-30T15:07:02.000+0000,224376,TU0AKgADa3z8sCyj/////////////////////////////////////////lv4yzXHK6DIKBci7BAyymgUrhYct9AkKSoNJMIMsqyGgrlIbAgcKCINphBMEDBAwg4QIEQZFFBlLMjovEcyOjNF8jwIGCDThBBA9NCIiIi0GEGdqgY= (truncated)
dbfs:/tmp/ocr/images/8230_014.2B.tif,2020-03-30T15:07:03.000+0000,218690,TU0AKgADQxCAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8250_007.2B.tif,2020-03-30T15:06:58.000+0000,139576,TU0AKgACDgaAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8200_006.2B.tif,2020-03-30T15:07:07.000+0000,116158,TU0AKgABsoyAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8200_014.2B.tif,2020-03-30T15:07:07.000+0000,116116,TU0AKgABsmKAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8200_015.2B.tif,2020-03-30T15:07:00.000+0000,80978,TU0AKgABKSCAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8202_024.2B.tif,2020-03-30T15:07:08.000+0000,75716,TU0AKgABJsj//////////////////////leBZkZgQRnBA04QPhA1oHRNxsG5XSxwiDkBrYQdAiMDPp0g7p0EHB06Tv6DutJ3ur/p/q/9+q8jHuW5p949P+1VXkIPpfxte3nHhpfxlczXvxXfrt9cPkOPWZCasSNIj8axhkzBNcE= (truncated)
dbfs:/tmp/ocr/images/8250_010.2B.tif,2020-03-30T15:07:04.000+0000,70142,TU0AKgAA/syAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)
dbfs:/tmp/ocr/images/8240_010.2B.tif,2020-03-30T15:06:57.000+0000,66458,TU0AKgABAp7////////////////////////////ybGER0dEYRZno4iOjCI5mEXziLTWs9EcGwjsjhCOBcjgXI4hHRdRERERERETurI6OZfERERERGIiP///+QGLfIDCQEMkvlMgYhMIjoIGZBYeW0owgyXBwgz5mSsNks4llnKQ= (truncated)
dbfs:/tmp/ocr/images/8250_004.2B.tif,2020-03-30T15:07:07.000+0000,63512,TU0AKgAA5OaAACBQOCQWDQeEQmFQuGQ2HQ+IRGJROKRWLReMRmNRuOR2PR+QSGRSOSSWTSeUSmVSuWS2XS+YTGZTOaTWbTecTmdTueT2fT+gUGhUOiUWjUekUmlUumU2nU+oVGpVOqVWrVesVmtVuuV2vV+wWGxWOyWWzWe0Wm0= (truncated)


In [28]:
# ACCESS_KEY = ""
# SECRET_KEY = ""
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", ACCESS_KEY)
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", SECRET_KEY)
# imagesPath = "s3a://dev.johnsnowlabs.com/ocr/datasets/news.2B/0/*.tif"
# imagesExampleDf = spark.read.format("binaryFile").load(imagesPath).cache()
# display(imagesExampleDf)

In [29]:
images_example_df.count()


Out[29]: 18

In [30]:
display_images(BinaryToImage().transform(images_example_df), limit=3)


Origin: dbfs:/tmp/ocr/images/8230_022.2B.tif 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr/images/8230_014.2B.tif 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr/images/8250_007.2B.tif 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1

In [31]:
result = pipeline().transform(images_example_df).cache()


In [32]:
display(result.select("text", "confidence"))


text confidence ‘Supermax’ home for feds’

FLORENCE — Beneath 34-foot-high
guard towers, workmen are putting the
finishing touches on ‘“Supermax,” also
known as “the Alcatraz of the Rockies.”

This is the part of the federal prison
complex that wiil house the nation’s most
notorious, most dangerous and most en-
dangered male prisoners.

It is intended to augment or possibly
supplant the “maximum-custody” prison
in Marion, Ili., which in turn replaced the
federal penal island of Alcatraz, in the
middle of San Francisco Bay, in 1963.

Because of the legendary heritage of Al-
catraz and, to a lesser degree, Marion, Su-
permax in Florence is already rich in folk-
lore, even though it won’t open for severai
more months.

Warden Bill Story, 54, is anxious to dis-
pel what he calls the public’s misunder-
standings about “U.S. Penitentiary/Ad-
ministrative Maximum,” Supermax’s
formal name.

Contrary to popular belief, Story said,
most of the 556 prisoners at Supermax
will not be kept in their single-cot cells 23
hours a day. Nor will all its prisoners, of-
ten referred to as ‘“‘the worst of the worst,”
be required to eat in their cells. And they
won'’t all be prohibited from attending re-
ligious services or other group activities.

Individual cells in Supermax’s “control
unit” will house 64 inmates 23 hours a day,
as will a separate 72-cell segregation unit,
which Story calls “a jail within a jail.” In-
mates held in this rigid system of confine-
ment will be permitted one hour a day out-
side their cells to exercise.

The other inmates will have access to
group activities and be permitted out of
their cells for anywhere from three to 12
hours a day.

Supermax also will have four VIP cells
should it receive an inmate with extraor-

 

 

 

 

 

 

 

 

about the plight of American Indians staged a peaceful vigil

Unlike Marion, where the worst prison-
ers are held in basement cells, all cells at

 

Still, “you can never say this is escape-
proof, because that could come back to
haunt you,” said Story.

 

The Denver Post / Helen H. Dav

PRISON PROTEST: About 125 protesters concerned outside the federal prison complex in this 1992 photo. One ¢

the major concerns was the treatment of inmates.

been manipulators all of their lives an
they can work people,” said Story, who
actively recruiting guards with extensiy

 88.58583813621884 Sherwood
Forest

rapidly

Merry men wouldn’t
recognize hideout

By William E. Schmidt
The New York Times

NOTTINGHAM, England — There isn’t
much left of Sherwood Forest, and what
there is, Robin Hood would hardly recog-
nize.

Farmland, pasture, coal mines and ur-
ban sprawl have claimed most of the
trees. Where there is greenwood, it con-
sists mostly of dense stands of Mediterra-
nean pine, imported and planted by the
government after World War I as a timber
cash crop.

But while Sherwood Forest is only a thin
wisp of its medieval past, its scattered
remnants are at the heart of national de-
bate over the future of Britain’s public
forest lands.

Prime Minister John Major’s Conserva-
tive government is contemplating whether
to turn over to private investors control of
4,355 square miles of woodland and heath,
nearly 5 percent of the nation’s land area.
The land is now managed by the Forestry
Commission, Britain’s largest landowner.

Proponents of free-market economics in
the Tory government argue that privatiza-
tion of Britain’s forests, like the earlier
privatization of airlines and utilities,
would draw fresh investment and help re-
new the moribund timber industry.

THE STORIES IT COULD TELL: The Major Oak in
Sherwood Forest, estimated by some to be 800 years old,

thusiasts has mounted a loud campaign,
warning that selling these lands would
sharply reduce public access to them.

“Now that the coal industry has col-
lapsed, we view our forests as the heart of
our county and the key to our future,” said
Gareth Broome, the forestry officer for
Nottinghamshire. “We obviously don’t
want to see them fal